In [1]:
%cd ../binfo1-work

/rna/seojin/proj/bioinformatics/bioinfo_term_proj/binfo1-work


In [2]:
!samtools mpileup ../binfo1-datapack1/CLIP-35L33G.bam > Seojin_Termproj/CLIP-35L33G.pileup
!wc -l CLIP-let7g.pileup

[mpileup] 1 samples in 1 input files
68548 CLIP-let7g.pileup


In [3]:
!head Seojin_Termproj/CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [4]:
import pandas as pd

In [5]:
pileup = pd.read_csv('Seojin_Termproj/CLIP-35L33G.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

/tmp/ipykernel_1103007/4078999649.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pileup = pd.read_csv('Seojin_Termproj/CLIP-35L33G.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])


,chrom,pos,_ref,count,basereads,quals
959697295,MU069435.1,31107,N,2,GG,BC
959697296,MU069435.1,31108,N,2,GG,B>
959697297,MU069435.1,31109,N,2,TT,@3
959697298,MU069435.1,31110,N,2,GG,B1
959697299,MU069435.1,31111,N,2,G$G$,@6


Leave only match(. or ,), mismatch(ACGT or acgt), deletion(* or */#) from basereads. I won't consider insertion(+)

In [6]:
import re

In [7]:
toremove = re.compile('[<>$^]|[^ATGCatgc]') 
pileup['filtered_basereads'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

In [14]:
pattern = re.compile(r'[-+][2-9]+[ACGTNacgtn]+')
cnt = 0
ct = 0 
for row in pileup.itertuples():
    if cnt == 1: 
        print(f"Cleaned Next row  : {row.pos}, {row.filtered_basereads}")
        print(f"Original Next row : {row.pos}, {row.basereads}")
        ct += 1
        if ct ==3 : 
            break
    if pattern.search(row.basereads):
        print(f"Cleaned  : {row.pos}, {row.filtered_basereads}")
        print(f"Original : {row.pos}, {row.basereads}")
        cnt += 1



Cleaned  : 3224571, AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Original : 3224571, AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA-2NNAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA-2NNAA-2NNAAAAAAAAAA-2NNAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Cleaned  : 3224575, CCCCCCCCCCCTCTCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
Original : 3224575, CCCCC-1NCCCCC-1NCTCTCC-1NCCCCCCCCCCC-1NC-1NCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC**CCCC-2NNCC-2NNCCCCCCCCCCCCCCC-2NNCCCCC-2NNCCCCCCCCC-2NNCCCCCCCCCCCC
Cleaned  : 3224581, CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
Or

KeyboardInterrupt: 

In [8]:
import numpy as np
def Shannon_entropy(match):
    num_A = match.upper().count('A')
    num_T = match.upper().count('T')
    num_G = match.upper().count('G')
    num_C = match.upper().count('C')

    # Calculate total number of valid bases
    total_bases = num_A + num_T + num_G + num_C

    # Calculate probabilities
    if total_bases > 0:
        p_A = num_A / total_bases
        p_T = num_T / total_bases
        p_G = num_G / total_bases
        p_C = num_C / total_bases

        # Calculate Shannon entropy
        entropy = 0
        for p in [p_A, p_T, p_G, p_C]:
            if p > 0:
                entropy -= p * np.log2(p)
    else:
        entropy = 0

    return entropy


In [9]:
def Calculate_Shannon_Entropy(pileup):
    for row in pileup.itertuples():
        shannon_entropy = Shannon_entropy(row.matches)
        print(shannon_entropy)
        pileup.at[row.Index, 'Shannon_entropy'] = shannon_entropy

In [ ]:
Calculate_Shannon_Entropy(pileup)